In [ ]:
import pandas as pd
import gensim
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import spacy
%run calculateSentence.ipynb
import preprocessing
pd.set_option('max_colwidth', 5000)

In [ ]:
#Importazione DB

data = pd.read_csv('DB/Preprocessed_data.csv')

data = data[data['processed_title'].notna()]

data

In [ ]:
#Importazione WordEmbeddings

word_2_vec_model = gensim.models.word2vec.Word2Vec.load('DB/word2vec_trained.bin')
word_2_vec_model

In [ ]:
def normalize(words):
    words = preprocessing.clear_text(words,"w2v")
    return words

In [ ]:
def w2v_result(txt, tag):
    stringSearch = normalize(txt)

    #print(stringSearch)

    numberResult = "10"

    #Viene vettorizzata la query dell'utente

    vectorSearch = np.array([questionToVec(stringSearch, word_2_vec_model)])
    
    #Carico il file con gli embeddings dei titoli sui quali effettuare la corrispondenza
    allTitleEmb = pd.read_csv('DB/titleEmbeddings.csv').values

    #Calcolo della similarità del coseno for le query e tutti i titoli

    similarityCosine = pd.Series(cosine_similarity(vectorSearch, allTitleEmb)[0])
    
    #Personalizzazione della misura di similarità

    #similarityCosine = similarityCosine*(1+0.4*data.overall_scores_norm + 0.1*(data.sentiment_polarity))
    similarityCosine = similarityCosine*(1+0.4*data.score)
    
    #Lista che conterrà coppie i,j con i indice e j score di similarità
    results = []
    tmp = 0 
    
    #Se tag è null vuol dire che siamo alla prima iterazione e non dobbiamo filtrare su nessun tag
    if tag == None :   
        #Una volta calcolati gli score di similarità rendo i 10 più alti
        for i,j in similarityCosine.nlargest(int(numberResult)).iteritems():
            #print(i, data.Body[i])
            #Filtro i tag da rendere come risultato, eliminando quelli che sono già presenti nella string search per facilitare prossimi passaggi 
            tags = data.iloc[[i]].tags.item().split('|')
            #print(tags)
            for t in tags:
                if t in stringSearch:
                    tags.remove(t)
                    
            results.append(tuple([data.iloc[[i]].id.values[0], j, tags]))
            print(results[tmp])
            tmp = tmp+1
    #Entriamo nell'else se abbiamo il tag che deve essere incluso nei nostri risultati, 
    #e eliminiamo i record che non lo contengono dai result 
    else : 
        
        for i,j in similarityCosine.nlargest(int(numberResult)).iteritems():
            
            if tag in data.iloc[[i]].tags:
                results.append(tuple([data.iloc[[i]].id.values[0], j, data.iloc[[i]].tags]))
                print(results[tmp])
                tmp = tmp+1
            
    
    return results

In [ ]:
w2v_result('problem loading csv',None)

In [ ]:
data.iloc[95375]